# This Notebook scrapes and cleans the Toronto Neighborhoods data from Wikipedia

## 1-  First the required libraries are imported

In [1]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup as bs

## 2-  This step opens the source html and scrape the data

In [2]:
# Open the HTML File from Wikipedia
htmlfile = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

# Reads the tables that contain the word 'Borough'
dfs = pd.read_html(htmlfile, match='Borough')

# Assign the first data frame extracted/scraped to a the data frame 'tndf' and displays the first 5 rows.
tndf = dfs[0]

tndf.head()

#The resulting dataframe 'tndf' has the requested format:

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 3-  This step cleans the data for 'Not assigned' Boroughs

In [3]:
# Find rows where the Borough is not assigned

indexNames = tndf[tndf['Borough'] == 'Not assigned'].index

# Drop rows where the Borough is not assigned

tndf.drop(indexNames, inplace=True)

# Reset index of the Data Frame

tndf.reset_index(drop=True, inplace=True)

# Displays the data frame first 5 rows

tndf.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 4- The below code looks for neighborhoods  that have the same postal code area (postal code area in more than one row) and combine them into one row. 
There are no such cases anymore on the Wikipedia page, but the code below would take care of new cases if they arise. 

In [10]:
# Find Postal Codes that are shown more than once

indexpc = tndf['Postal Code'].value_counts() > 1 #I dentify the Postal Codes that are more than one time on the data frame tndf 

pc = indexpc[indexpc] 

pci = pc.index # Holds a list of the Postal Codes that are more than once on the data frame tndf

# For each Postal Code that is shown more than once get the rows index

for i in pci:
   

    udf = tndf[tndf['Postal Code']==i]

    indexdf = udf.index
    
    countindex = len(udf.index)
    

    # Reset the variable for concatenating the Neighborhood names and the counter for updating the data frame rows     
    anb = ""
    count = 1

    for j in indexdf:
        
       
        nb = tndf['Neighborhood'].loc[j]
        if count == 1:
            anb = nb
        else:    
            anb = nb+", "+anb  # Concatenates neighborhoods that share the same Postal Code Area
        if count < countindex:
            tndf.drop(j, inplace=True) # Deletes rows for neighborhoods that share the same Postal Code Area unless is the last row
        elif count == countindex:
            tndf['Neighborhood'].loc[j] = anb # Updates the Neiborhood with the comma separated list of neighborhoods that share the same postal code area
        
        count = count + 1


# Reset index of the Data Frame

tndf.reset_index(drop=True, inplace=True)

tndf.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 4- The fourth step is to Clean the data for 'Not assigned' Neighborhoods assigning the Borough Name.
No cases are currently on the data, but in case there are the following code will fix it.

In [7]:
# Find rows where the Neighborhood is 'Not assigned'

indexNames2 = tndf[tndf['Neighborhood'] == 'Not assigned'].index

# Assign the Borough name to the Neighborhood where the Neighborhood is not assigned

for i in indexNames2:

    b = tndf['Borough'].loc[i]
    tndf['Neighborhood'].loc[i] = b  


## 5- Using .shape method to print the number of rows of the dataframe

In [9]:
tndf.shape

(103, 3)